In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys

if 'google.colab' in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')
  path_prefix = '/content/drive/MyDrive/Great Learning/DBA/Walsh-MS-QM640CapstoneProject/'
  print("Running in Google Colab.")
else:
  path_prefix = './'
  print("Not running in Google Colab.")

print(f"Using path prefix: {path_prefix}")

Not running in Google Colab.
Using path prefix: ./


In [74]:
df = pd.read_excel(f"{path_prefix}scored/reviews_predicted_eng.xlsx")
print(df.shape)
df.head()

(53719, 20)


,reviewId,reviewContent,rating,reviewedAt,replyContent,repliedAt,store,app_name,at_year,at_month,country,at_calc,at_eta_date,reply_in,reply_eta,sla_status,responded_by,language_pred,department_pred,sentiment_pred
0,a2393404-7755-4564-add7-b3029159bb71,the application now for 4 days is not opening ...,1,2025-07-01 02:04:14,Sorry to hear you're having trouble with the a...,2025-07-01 12:49:59,Google Play Store,ADCB,2025,7,NaN,2025-07-01 08:00:00,2025-07-01 20:00:00,4.833056,0.298623,SLA Met,A.E,en,Application,Negative
1,f521326b-0ffb-4620-8995-b04eeee01c8f,normal,3,2025-06-30 22:17:54,NaN,NaT,Google Play Store,ADCB,2025,6,NaN,2025-07-01 08:00:00,2025-07-01 20:00:00,NaN,-1000.000000,Not Responded,NaN,en,General,Neutral
2,6262b57b-7c21-4f6b-9a6a-b4fa8d60b395,good,5,2025-06-30 21:42:32,"Hello, thank you for the encouraging star rati...",2025-07-01 12:49:03,Google Play Store,ADCB,2025,6,NaN,2025-07-01 08:00:00,2025-07-01 20:00:00,4.817500,0.299271,SLA Met,A.E,en,General,Positive
3,1780c47d-38ad-4563-92c3-c381b717171f,good thank you,5,2025-06-30 19:14:18,"Hello, thank you for the encouraging star rati...",2025-07-01 12:49:00,Google Play Store,ADCB,2025,6,NaN,2025-06-30 19:14:18,2025-07-01 19:14:18,5.578333,0.267569,SLA Met,A.E,en,General,Positive
4,5083ac69-2298-45ee-9b96-1912e10dfed6,Dear adcb bank i had a trouble on the Applicat...,5,2025-06-30 19:08:16,Sorry to hear you're having trouble with the a...,2025-07-01 12:49:52,Google Play Store,ADCB,2025,6,NaN,2025-06-30 19:08:16,2025-07-01 19:08:16,5.693333,0.262778,SLA Met,A.E,en,Application,Negative


In [75]:
# Rename columns for clarity in RAG pipeline context
df = df.rename(columns={
    'reviewId': 'review_id',
    'reviewContent': 'review_text',
    'rating': 'star_rating',
    'reviewedAt': 'review_date',
    'replyContent': 'reply_text',
    'repliedAt': 'reply_date',
    'store': 'app_store',
    'app_name': 'app_name',
    'at_year': 'year',
    'at_month': 'month',
    'country': 'country',
    'at_calc': 'calc_date',
    'at_eta_date': 'eta_date',
    'reply_in': 'reply_time_hrs',
    'reply_eta': 'reply_eta',
    'sla_status': 'sla_status',
    'responded_by': 'responded_by',
    'language_pred': 'language',
    'department_pred': 'department',
    'sentiment_pred': 'sentiment'
})

print("Columns renamed for RAG pipeline clarity.")
df.head()

Columns renamed for RAG pipeline clarity.


,review_id,review_text,star_rating,review_date,reply_text,reply_date,app_store,app_name,year,month,country,calc_date,eta_date,reply_time_hrs,reply_eta,sla_status,responded_by,language,department,sentiment
0,a2393404-7755-4564-add7-b3029159bb71,the application now for 4 days is not opening ...,1,2025-07-01 02:04:14,Sorry to hear you're having trouble with the a...,2025-07-01 12:49:59,Google Play Store,ADCB,2025,7,NaN,2025-07-01 08:00:00,2025-07-01 20:00:00,4.833056,0.298623,SLA Met,A.E,en,Application,Negative
1,f521326b-0ffb-4620-8995-b04eeee01c8f,normal,3,2025-06-30 22:17:54,NaN,NaT,Google Play Store,ADCB,2025,6,NaN,2025-07-01 08:00:00,2025-07-01 20:00:00,NaN,-1000.000000,Not Responded,NaN,en,General,Neutral
2,6262b57b-7c21-4f6b-9a6a-b4fa8d60b395,good,5,2025-06-30 21:42:32,"Hello, thank you for the encouraging star rati...",2025-07-01 12:49:03,Google Play Store,ADCB,2025,6,NaN,2025-07-01 08:00:00,2025-07-01 20:00:00,4.817500,0.299271,SLA Met,A.E,en,General,Positive
3,1780c47d-38ad-4563-92c3-c381b717171f,good thank you,5,2025-06-30 19:14:18,"Hello, thank you for the encouraging star rati...",2025-07-01 12:49:00,Google Play Store,ADCB,2025,6,NaN,2025-06-30 19:14:18,2025-07-01 19:14:18,5.578333,0.267569,SLA Met,A.E,en,General,Positive
4,5083ac69-2298-45ee-9b96-1912e10dfed6,Dear adcb bank i had a trouble on the Applicat...,5,2025-06-30 19:08:16,Sorry to hear you're having trouble with the a...,2025-07-01 12:49:52,Google Play Store,ADCB,2025,6,NaN,2025-06-30 19:08:16,2025-07-01 19:08:16,5.693333,0.262778,SLA Met,A.E,en,Application,Negative


In [72]:
df['sla_status'].unique()

array(['SLA Met', 'Not Responded', 'SLA Missed'], dtype=object)

In [5]:
%pip install sentence-transformers faiss-cpu transformers torch

In [17]:
# Install required libraries (uncomment if running for the first time)
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline
import torch
import openai
import os
import re
print('Libraries imported for RAG pipeline.')

Libraries imported for RAG pipeline.


In [8]:
# Generate embeddings for review_text using OpenAI text-embedding-3-large
from tqdm import tqdm

openai_embed_model = "text-embedding-3-large"
openai_client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Drop NA reviews for embedding
texts = df['review_text'].fillna('').tolist()

# Batch embedding for efficiency
def get_openai_embeddings(texts, batch_size=100):
    all_embeds = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = [str(t) if t is not None else "" for t in texts[i:i+batch_size]]
        # Remove empty strings (OpenAI API does not accept them)
        batch = [t for t in batch if t.strip()]
        if not batch:
            continue
        response = openai_client.embeddings.create(
            input=batch,
            model=openai_embed_model
        )
        batch_embeds = [d.embedding for d in response.data]
        all_embeds.extend(batch_embeds)
    return np.array(all_embeds)

embeddings = get_openai_embeddings(texts)

# Save embeddings to disk for reuse
embeddings_path = f"{path_prefix}scored/review_embeddings_openai.npy"
np.save(embeddings_path, embeddings)
print(f"Embeddings saved to {embeddings_path}. To reload, use: np.load(embeddings_path)")

100%|██████████| 538/538 [19:40<00:00,  2.19s/it]


Embeddings saved to ./scored/review_embeddings_openai.npy. To reload, use: np.load(embeddings_path)


In [9]:
# Reload embeddings from disk
embeddings_path = f"{path_prefix}scored/review_embeddings_openai.npy"
embeddings = np.load(embeddings_path)
print(f"Embeddings reloaded from {embeddings_path}. Shape: {embeddings.shape}")

Embeddings reloaded from ./scored/review_embeddings_openai.npy. Shape: (53719, 3072)


In [10]:
# Build FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print(f"FAISS index built with {len(texts)} reviews using OpenAI text-embedding-3-large.")

FAISS index built with 53719 reviews using OpenAI text-embedding-3-large.


In [70]:
client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Hybrid RAG pipeline: intent detection, semantic & structured (pandas) search, LLM-assisted text-to-pandas

def detect_intent(prompt):
    """Very simple intent detection: if prompt contains keywords, treat as structured; else semantic."""
    structured_keywords = ["show", "list", "count", "number of", "average", "sum", "group by", "table", "sql", "pandas"]
    for kw in structured_keywords:
        if kw in prompt.lower():
            return "structured"
    return "semantic"

# Column descriptions for the DataFrame
df_column_descriptions = {
    'review_id': 'Unique review identifier',
    'review_text': 'Text of the customer review',
    'star_rating': 'Star rating given by the user (numeric)',
    'review_date': 'Date of the review (datetime)',
    'reply_text': 'Text of the reply from support (if any)',
    'reply_date': 'Date of the reply (datetime, if any)',
    'app_store': 'App store source (Google Play or Apple)',
    'app_name': 'Name of the app',
    'year': 'Year of the review',
    'month': 'Month of the review',
    'country': 'Country of the reviewer',
    'calc_date': 'Calculated date field (if present)',
    'eta_date': 'Estimated time of arrival date (if present)',
    'reply_time_days': 'Reply time in days',
    'reply_eta': 'Reply ETA (if present)',
    'sla_status': 'SLA status for the reply',
    'responded_by': 'Name or ID of the responder',
    'language': 'Predicted language of the review',
    'department': 'Predicted department for the review',
    'sentiment': 'Predicted sentiment label',
}

# LLM-assisted text-to-pandas code generation
def generate_pandas_code(prompt, df_name="df"):
    # Get unique values for key columns
    sentiment_values = df['sentiment'].unique().tolist() if 'sentiment' in df.columns else []
    department_values = df['department'].unique().tolist() if 'department' in df.columns else []
    sla_status_values = df['sla_status'].unique().tolist() if 'sla_status' in df.columns else []
    language_values = df['language'].unique().tolist() if 'language' in df.columns else []
    app_name_values = df['app_name'].unique().tolist() if 'app_name' in df.columns else []
    app_store_values = df['app_store'].unique().tolist() if 'app_store' in df.columns else []

    column_info = "\n".join([f"{col}: {desc}" for col, desc in df_column_descriptions.items()])
    value_info = (
        f"sentiment possible values: {sentiment_values}\n"
        f"department possible values: {department_values}\n"
        f"sla_status possible values: {sla_status_values}\n"
        f"language possible values: {language_values}\n"
        f"app_name possible values: {app_name_values}\n"
        f"app_store possible values: {app_store_values}"
    )
    system_msg = (
        "You are a Python data analyst. Given a user request and a pandas DataFrame called 'df', "
        "write a single line of pandas code to answer the request. Only output the code, no explanation.\n"
        f"DataFrame columns and descriptions:\n{column_info}\n"
        f"{value_info}"
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": prompt}
        ],
        max_tokens=128,
        temperature=0.0
    )
    code = response.choices[0].message.content.strip()
    return code

def hybrid_rag(prompt, k=10):
    intent = detect_intent(prompt)
    if intent == "semantic":
        # Use semantic RAG pipeline
        print("Using semantic RAG pipeline")
        query_emb = openai_client.embeddings.create(
            input=[prompt],
            model=openai_embed_model
        ).data[0].embedding
        query_emb = np.array(query_emb, dtype=np.float32).reshape(1, -1)
        D, I = index.search(query_emb, k)
        retrieved_texts = [texts[i] for i in I[0]]
        context = "\n".join(retrieved_texts)
        rag_prompt = f"Context:\n{context}\n\nQuestion: {prompt}\nAnswer:"
        strict_system_msg = (
            "You are a helpful assistant for customer experience analytics. "
            "Only answer based on the provided context you may summarize or synthesize from the context, but do not hallucinate."
        )
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": strict_system_msg},
                {"role": "user", "content": rag_prompt}
            ],
            max_tokens=256,
            temperature=0.2
        )
        answer = response.choices[0].message.content.strip()
        if answer.strip().lower().startswith("i don't know based on the search results"):
            print("\nNo answer found in search results. Here are the top retrieved review excerpts:\n")
            print(context)
        return answer
    else:
        print("Using structured RAG pipeline")
        # Use LLM to generate pandas code, then execute and return result
        code = generate_pandas_code(prompt)
        print(f"Generated pandas code: {code}")
        # Remove markdown code block formatting if present
        code = "\n".join([line for line in code.splitlines() if not line.strip().startswith("```")]).strip()
        try:
            result = eval(code)
            # Handle empty results
            if hasattr(result, 'empty') and result.empty:
                result = "No results found for this query."
            elif isinstance(result, (pd.Series, pd.DataFrame)) and result.shape[0] == 0:
                result = "No results found for this query."
        except Exception as e:
            result = f"Error executing code: {e}"
        return result

print("Hybrid RAG pipeline ready: use hybrid_rag(prompt) for both semantic and structured queries.")

Hybrid RAG pipeline ready: use hybrid_rag(prompt) for both semantic and structured queries.


In [49]:
# Example: Operational CX query using RAG pipeline
sample_query = "show departments that had most complaints with results?"
rag_answer = hybrid_rag(sample_query)
print(f"Q: {sample_query}\nA: {rag_answer}")

Using structured RAG pipeline
Generated pandas code: ```python
df[df['sentiment'] == 'Negative']['department'].value_counts()
```
Q: show departments that had most complaints with results?
A: department
Application          9130
General              6296
CustomerService      2899
Cards                 680
TransfersPayments     557
Fraud                 323
FeesCharges           311
CASA                  219
Offers                200
Deposits                9
Investment              6
Name: count, dtype: int64


In [51]:
# Example: Operational CX query using RAG pipeline
sample_query = "show banks that had most compliant SLA Status with results?"
rag_answer = hybrid_rag(sample_query)
print(f"Q: {sample_query}\nA: {rag_answer}")

Using structured RAG pipeline
Generated pandas code: ```python
df[df['sla_status'] == 'SLA Met']['app_name'].value_counts()
```
Q: show banks that had most compliant SLA Status with results?
A: app_name
ADCB        8175
Mashreq     6432
ENBD        5214
ADIB        2659
RAKBANK     1365
FAB          805
DIB          687
SC Bank      279
EI Bank       62
Citibank      58
Name: count, dtype: int64


In [60]:
# Example: Semantic query using RAG pipeline (should trigger semantic search, not structured)
semantic_query = "What are some common themes or issues mentioned by users in their reviews?"
rag_semantic_answer = hybrid_rag(semantic_query)
print(f"Q: {semantic_query}\nA: {rag_semantic_answer}")

Using semantic RAG pipeline
Q: What are some common themes or issues mentioned by users in their reviews?
A: Some common themes or issues mentioned by users in their reviews include a lack of features, poor performance, compatibility issues, limited functionality, and frequent maintenance issues. Additionally, users have expressed frustration with the app being anti-user friendly and requiring reactivation after certain actions.


In [71]:
# Example: Semantic query using RAG pipeline (should trigger semantic search, not structured)
semantic_query = "What are some common issues mentioned by users in their reviews related to cards?"
rag_semantic_answer = hybrid_rag(semantic_query)
print(f"Q: {semantic_query}\nA: {rag_semantic_answer}")

Using semantic RAG pipeline
Q: What are some common issues mentioned by users in their reviews related to cards?
A: Some common issues mentioned by users in their reviews related to cards include:

1. Bugs and missing features in the app, such as certain cards not appearing on the home page and the inability to access functionalities like freezing the card or viewing transaction history.
2. Delays and lack of response to credit card closure requests.
3. Hidden charges and discrepancies between what bank staff promise and what is delivered.
4. Long wait times for customer support, often taking 10 to 20 minutes to connect.
5. Difficulty in locating key card details like statement date, payment date, last payment amount, and balance.
6. Lack of integration with mobile payment platforms like Samsung Pay and Google Pay.
7. App crashes and errors, particularly when trying to manage cards.
8. Limited banking options and delays in updating card credits.
9. Difficulty in understanding the actua